In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as datasets
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from copy import deepcopy

In [5]:
def load_model(file_path = "./models/glove.twitter.27B.50d.txt"):
    glove = {}
    with open(file_path, "r", encoding='utf-8') as f:
        for lines in f:
            items = lines.split()
            if len(items) != 51:
                continue
            else:
                word_vector = []
                for i in range(1,51):
                    word_vector.append(float(items[i]))
                glove[items[0]] = word_vector
    return glove
GloVe = load_model()

FileNotFoundError: [Errno 2] No such file or directory: './models/glove.twitter.27B.50d.txt'

In [3]:
contractions_dict = {
    "i'm" : "i am",
    "i'll" : "i will",
    "i'd" : "i would",
    "i've" : "i have",
    "you're" : "you are",
    "you'll" : "you will",
    "you'd" : "you would",
    "you've" : "you have",
    "she's" : "she is",
    "she'll" : "she will",
    "he's" : "he is",
    "he'll" : "he will",
    "he'd" : "he would",
    "they're" : "they are",
    "they'll" : "they will",
    "they'd" : "they would",
    "that's" : "that is",
    "that'll" : "that will",
    "that'd" : "that would",
    "who's" : "who is",
    "who'll" : "who will",
    "who'd" : "who would",
    "what's" : "what is",
    "what're" : "what are",
    "what'll" : "what will",
    "what'd" : "what would",
    "where's" : "where is",
    "where'll" : "where will",
    "where'd" : "where would",
    "when's" : "when is",
    "when'll" : "when will",
    "when'd" : "when would",
    "why's" : "why is",
    "why'll" : "why will",
    "why'd" : "why would",
    "how's" : "how is",
    "how'll" : "how will",
    "how'd" : "how would",
    "would've" : "would have",
    "should've" : "should have",
    "could've" : "could have",
    "might've" : "might have",
    "must've" : "must have",
    "isn't" : "is not",
    "aren't" : "are not",
    "wasn't" : "was not",
    "weren't" : "were not",
    "haven't" : "have not",
    "hasn't" : "has not",
    "hadn't" : "had not",
    "won't" : "will not",
    "wouldn't" : "would not",
    "don't" : "do not",
    "doesn't" : "does not",
    "didn't" : "did not",
    "can't" : "cannot",
    "couldn't" : "could not",
    "shouldn't" : "should not",
    "mightn't" : "might not",
    "mustn't" : "must not"
}

def contractionfunction(s):
    if s in contractions_dict.keys():
        return contractions_dict[s]
    return s

In [4]:
char_replace = {",",".","/",";","'","[","]","\\","!","@","#","$","%","^","&","*","(",")","-","_","=","+","<",">","?",":","\"","{","}","|"}

In [122]:
def load_train(file_path = "./data/emotion_data_train.txt"):
    train_x = []
    train_y = []
    with open(file_path, "r", encoding='utf-8') as f:
        for lines in f:
            y_x = lines.split("|")
            if len(y_x) < 2:
                continue
            train_y.append(y_x[0])
            train_x.append(y_x[1])
    return train_x, train_y

def load_dev(file_path = "./data/emotion_data_test.txt"):
    dev_x = []
    dev_y = []
    with open(file_path, "r", encoding='utf-8') as f:
        for lines in f:
            y_x = lines.split("|")
            if len(y_x) < 2:
                continue
            dev_y.append(y_x[0])
            dev_x.append(y_x[1])
    return dev_x, dev_y

In [123]:
train_x_raw, train_y_raw = load_train()
dev_x_raw, dev_y_raw = load_dev()

In [124]:
print(len(train_x_raw))
print(len(train_y_raw))
print(len(dev_x_raw))
print(len(dev_y_raw))

7016
7016
500
500


In [125]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
#Data preprocess:
def preprocess(sentence_list):
    for i in range(len(sentence_list)):
        sentence_list[i] = sentence_list[i].lower()
        for char in char_replace:
            if char in sentence_list[i]:
                sentence_list[i] = sentence_list[i].replace(char, " ")
        sentence_list[i] = ' '.join([word for word in sentence_list[i].split() if word not in (stop)])
        sentence_list[i] = sentence_list[i].split()
    return sentence_list

In [126]:
train_x_raw = preprocess(train_x_raw)
dev_x_raw = preprocess(dev_x_raw)

In [127]:
y_count = {}
for y in train_y_raw:
    if y in y_count:
        y_count[y] += 1
    else:
        y_count[y] = 1

In [128]:
y_count

{'joy': 1020,
 'fear': 1002,
 'anger': 1005,
 'sadness': 1009,
 'disgust': 995,
 'shame': 1000,
 'guilt': 985}

In [129]:
def sort_key(a):
    return a[1]

In [130]:
label_list = []
for key in y_count:
    item = y_count[key]
    label_list.append((key,item))

In [131]:
label_list.sort(reverse=True,key=sort_key)

In [132]:
label_list

[('joy', 1020),
 ('sadness', 1009),
 ('anger', 1005),
 ('fear', 1002),
 ('shame', 1000),
 ('disgust', 995),
 ('guilt', 985)]

In [133]:
label2id = {}
i = 0
for label in label_list:
    label2id[label[0]] = i
    label2id[i] = label[0]
    i+=1

In [134]:
label2id

{'joy': 0,
 0: 'joy',
 'sadness': 1,
 1: 'sadness',
 'anger': 2,
 2: 'anger',
 'fear': 3,
 3: 'fear',
 'shame': 4,
 4: 'shame',
 'disgust': 5,
 5: 'disgust',
 'guilt': 6,
 6: 'guilt'}

In [135]:
def get_type_glove(unk):
    digits = 0
    for c in unk:
        if c.isdigit():
            digits += 1
    df = digits/len(unk)
    if unk.isdigit():
        return 1.0
    elif df > 0.5:
        return 2.0
    elif digits>0:
        return 3.0
    else:
        return 0.0

In [136]:
UNK = "< UNK >"
GloVe[UNK] = np.random.uniform(-0.25, 0.25, 200).tolist()

In [137]:
max_sentence_length = 0
for sentence in train_x_raw:
    if len(sentence) > max_sentence_length:
        max_sentence_length = len(sentence)

In [138]:
max_sentence_length

78

In [139]:
class average_meter(object):
    '''Computes and stores the average and current value
    '''
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [140]:
class emotion_dataset(datasets.Dataset):
    
    def __init__(self, word_lists, label_lists):
        self.word_lists = word_lists
        self.label_lists = label_lists

    def __getitem__(self, index):
        return self.word_lists[index], self.label_lists[index]
    
    def __len__(self):
        return len(self.word_lists)

In [196]:
def data_process(data):
    b_size = len(data)
    x_tensor = torch.zeros(b_size, 30, 201)
    y_tensor = torch.zeros(b_size).long()
    
    for b_index in range(b_size):
        x = data[b_index][0]
        y = data[b_index][1]
        for xy_index in range(min(len(x),30)):
            word = x[xy_index]
            if word in GloVe:
                x_vector = deepcopy(GloVe[word])
                one_more = get_type_glove(word)
                x_vector.append(one_more)
                _x = torch.FloatTensor(x_vector)
                x_tensor[b_index][xy_index] = _x
            else:
                unk_vector = deepcopy(GloVe[UNK])
                one_more = get_type_glove(word)
                unk_vector.append(one_more)
                _unk = torch.FloatTensor(unk_vector)
                x_tensor[b_index][xy_index] = _unk
        y_tensor[b_index] = label2id[y]
    data_len = []
    for xy in data:
        data_len.append(len(xy[0]))
    return x_tensor, y_tensor, data_len

In [226]:
TRAIN_BATCH_SIZE = 512
DEV_BATCH_SIZE = 512

In [227]:
train_dataset = emotion_dataset(train_x_raw,train_y_raw)
dev_dataset = emotion_dataset(dev_x_raw,dev_y_raw)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=False, collate_fn=data_process)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=DEV_BATCH_SIZE, shuffle=False, collate_fn=data_process)

In [228]:
class CNN_Text(nn.Module):
    
    def __init__(self):
        super(CNN_Text, self).__init__()
        filter_sizes = [1,2,3,5]
        num_filters = 36
        n_classes = 7
        self.convs1 = nn.ModuleList([nn.Conv2d(1, num_filters, (K, 201)) for K in filter_sizes])
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(len(filter_sizes)*num_filters, n_classes)


    def forward(self, x):
        x = x.unsqueeze(1)  
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1] 
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  
        x = torch.cat(x, 1)
        x = self.dropout(x)  
        logit = self.fc1(x) 
        return logit

In [229]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
n_epochs = 50
model = CNN_Text().to(DEVICE)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)

In [230]:
def train(model, train_loader, criterion, optimizer, epoch):
    train_loss = average_meter()
    model.train()
    total = len(train_loader)
    for i in train_loader:
        x = i[0].to(DEVICE)
        y = i[1].to(DEVICE)
        l = i[2]
        pred = model.forward(x)
        optimizer.zero_grad()
        loss = criterion(pred, y).to(DEVICE)
        train_loss.update(loss.item(),x.size(0))
        loss.backward()
        optimizer.step()
    return train_loss.avg
        
def validate(model, dev_loader, criterion, optimizer, epoch):
    b_size = DEV_BATCH_SIZE
    valid_loss = average_meter()
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        step = 0
        for i in dev_loader:
            x = i[0].to(DEVICE)
            y = i[1].to(DEVICE)
            l = i[2]
            pred = model.forward(x)
            loss = criterion(pred, y).to(DEVICE)
            valid_loss.update(loss.item(),x.size(0))
            pred = torch.max(pred, 1)[1]
            correct += (pred == y).float().sum()
            total += y.shape[0]
            step += 1
    return valid_loss.avg, correct/total

In [231]:
best_acc = 0
acc_model = None

for epoch in range(n_epochs):
    torch.cuda.empty_cache()
    train_loss = train(model, train_dataloader, criterion, optimizer, epoch)
    torch.cuda.empty_cache()
    scheduler.step()
    valid_loss, valid_acc = validate(model, dev_dataloader, criterion, optimizer, epoch)
    if valid_acc > best_acc:
        acc_model = deepcopy(model)
        best_acc = valid_acc
    print("{}%, Epoch {}, train loss {:.8f}, validate loss {:.8f}, acc {:.8f}".format(100*(epoch+1)/n_epochs,str(epoch).zfill(2),train_loss,valid_loss,valid_acc))

KeyboardInterrupt: 

In [154]:
def sentence_to_vec(s):
    s = s.lower()
    for char in char_replace:
        if char in s:
            s = s.replace(char, " ")
    s = ' '.join([word for word in s.split() if word not in (stop)])
    s = s.split()
    s_tensor = torch.zeros(1, 80, 201)
    for i in range(len(s)):
        word = s[i]
        if word in GloVe:
            x_vector = deepcopy(GloVe[word])
            one_more = get_type_glove(word)
            x_vector.append(one_more)
            _x = torch.FloatTensor(x_vector)
            s_tensor[0][i] = _x
        else:
            unk_vector = deepcopy(GloVe[UNK])
            one_more = get_type_glove(word)
            unk_vector.append(one_more)
            _unk = torch.FloatTensor(unk_vector)
            s_tensor[0][i] = _unk
    return s_tensor

In [155]:
sentence = "I feel very happy"
s = sentence_to_vec(sentence).to(DEVICE)
pred_s = model.forward(s)
pred_y = torch.max(pred_s, 1)[1]
print(pred_s)
print(pred_y)

tensor([[ 7.4147,  0.2790, -2.4521, -4.2422, -2.1910, -3.6526, -0.8217]],
       device='cuda:0', grad_fn=<AddmmBackward>)
tensor([0], device='cuda:0')


In [156]:
label2id

{'joy': 0,
 0: 'joy',
 'sadness': 1,
 1: 'sadness',
 'anger': 2,
 2: 'anger',
 'fear': 3,
 3: 'fear',
 'shame': 4,
 4: 'shame',
 'disgust': 5,
 5: 'disgust',
 'guilt': 6,
 6: 'guilt'}

In [157]:
model_1=torch.load("emotional_cnn.pt")

In [158]:
dev_predict_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=len(dev_dataset), shuffle=False, collate_fn=data_process)

In [167]:
correct = 0
total = 0
model_1.eval()
with torch.no_grad():
    step = 0
    for i in dev_predict_loader:
        x = i[0].to(DEVICE)
        y = i[1].to(DEVICE)
        l = i[2]
        pred = model_1.forward(x)
        pred = torch.max(pred, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        step += 1
acc = correct/total

In [168]:
acc

tensor(0.9920, device='cuda:0')

In [169]:
y

tensor([3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 0, 3, 2, 1, 5,
        4, 6, 0, 3, 2, 3, 2, 1, 5, 4, 6, 0, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0,
        3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2,
        0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 5, 4, 6, 0, 3, 2, 5,
        4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0,
        3, 2, 1, 5, 4, 6, 0, 3, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5,
        4, 6, 0, 3, 2, 1, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4,
        6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 1, 5, 4, 6, 0,
        3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1,
        5, 4, 6, 0, 3, 2, 1, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0,
        3, 2, 1, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 4, 6,
        0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 3, 2, 1, 5,
        1, 5, 4, 6, 0, 3, 2, 1, 5, 4, 6,

In [166]:
pred

tensor([3, 2, 1, 5, 4, 6, 0, 4, 5, 3, 4, 4, 5, 2, 3, 5, 4, 5, 5, 0, 3, 2, 1, 5,
        3, 4, 2, 3, 6, 0, 1, 5, 5, 5, 2, 0, 1, 5, 4, 5, 0, 5, 2, 1, 2, 6, 3, 3,
        3, 2, 2, 2, 4, 2, 0, 3, 2, 6, 5, 3, 4, 6, 0, 6, 0, 4, 5, 4, 6, 0, 3, 1,
        0, 3, 3, 0, 3, 3, 4, 6, 3, 1, 1, 5, 0, 2, 2, 5, 4, 5, 3, 6, 0, 3, 6, 5,
        4, 1, 3, 3, 2, 1, 5, 4, 1, 0, 3, 6, 1, 5, 4, 6, 0, 3, 5, 3, 3, 0, 2, 0,
        3, 2, 1, 5, 3, 6, 0, 3, 3, 3, 4, 2, 0, 3, 2, 1, 4, 1, 6, 0, 3, 2, 1, 3,
        0, 2, 0, 3, 6, 1, 2, 5, 0, 5, 5, 0, 3, 2, 1, 2, 6, 4, 0, 3, 2, 6, 5, 2,
        6, 1, 1, 4, 1, 1, 6, 2, 3, 3, 2, 1, 5, 5, 3, 0, 3, 2, 1, 1, 0, 6, 2, 2,
        2, 4, 1, 5, 4, 2, 0, 3, 2, 1, 5, 3, 6, 0, 2, 2, 1, 4, 2, 3, 0, 3, 2, 1,
        1, 4, 6, 0, 5, 5, 1, 4, 4, 0, 3, 2, 0, 4, 4, 4, 0, 3, 0, 1, 2, 4, 6, 0,
        3, 2, 1, 0, 2, 4, 4, 0, 6, 2, 1, 0, 4, 6, 0, 3, 4, 1, 2, 4, 2, 0, 0, 2,
        0, 3, 5, 1, 2, 4, 5, 0, 3, 0, 0, 4, 3, 6, 0, 3, 5, 1, 5, 4, 3, 2, 5, 4,
        3, 6, 4, 6, 0, 3, 2, 1, 2, 4, 4,

In [208]:
class Emotion_GloVe(nn.Module):
    def __init__(self):
        super(Emotion_GloVe, self).__init__()
        self.lstm = nn.LSTM(input_size = 201,
                                 hidden_size = 256,
                                 num_layers = 1,
                                 bidirectional=True)
        self.fc1 = nn.Linear(2* 256, 7)
    
    def forward(self, x):
        h0 = torch.zeros(2, 30, 256).requires_grad_().to(DEVICE)
        c0 = torch.zeros(2, 30, 256).requires_grad_().to(DEVICE)
        output, h = self.lstm(x, (h0, c0))
        output = output[:, -1, :]
        output = F.dropout(output, 0.5)
        output = self.fc1(output)
        output = F.softmax(output, dim=1)
        return output

In [209]:
n_epochs = 20
model = Emotion_GloVe().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [210]:
best_acc = 0
acc_model = None

for epoch in range(n_epochs):
    torch.cuda.empty_cache()
    train_loss = train(model, train_dataloader, criterion, optimizer, epoch)
    torch.cuda.empty_cache()
    scheduler.step()
    valid_loss, valid_acc = validate(model, dev_dataloader, criterion, optimizer, epoch)
    if valid_acc > best_acc:
        acc_model = deepcopy(model)
        best_acc = valid_acc
    print("{}%, Epoch {}, train loss {:.8f}, validate loss {:.8f}, acc {:.8f}".format(100*(epoch+1)/n_epochs,str(epoch).zfill(2),train_loss,valid_loss,valid_acc))

5.0%, Epoch 00, train loss 2.01010158, validate loss 2.01973648, acc 0.14600001
10.0%, Epoch 01, train loss 2.02004334, validate loss 2.01705277, acc 0.14800000
15.0%, Epoch 02, train loss 2.02254717, validate loss 2.01742150, acc 0.14800000
20.0%, Epoch 03, train loss 2.02259445, validate loss 2.01728049, acc 0.14800000
25.0%, Epoch 04, train loss 2.02259930, validate loss 2.01742007, acc 0.14800000
30.0%, Epoch 05, train loss 2.02259353, validate loss 2.01742150, acc 0.14800000
35.0%, Epoch 06, train loss 2.02260491, validate loss 2.01742139, acc 0.14800000
40.0%, Epoch 07, train loss 2.02260478, validate loss 2.01742149, acc 0.14800000
45.0%, Epoch 08, train loss 2.02259975, validate loss 2.01742152, acc 0.14800000
50.0%, Epoch 09, train loss 2.02260226, validate loss 2.01742149, acc 0.14800000
55.0%, Epoch 10, train loss 2.02260026, validate loss 2.01742355, acc 0.14800000
60.0%, Epoch 11, train loss 2.02259781, validate loss 2.01742149, acc 0.14800000
65.0%, Epoch 12, train loss 2

In [256]:
class GRUNet(nn.Module):
    def __init__(self):
        super(GRUNet, self).__init__()
        self.h_size = 256
        self.rnn = nn.GRU(201, 256, batch_first = True)
        self.fc = nn.Linear(256, 7)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), self.h_size).to(DEVICE)
        out, hidden = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out, hidden

In [257]:
n_epochs = 50
model = GRUNet().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.3)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [258]:
def train(model, train_loader, criterion, optimizer, epoch):
    train_loss = average_meter()
    model.train()
    total = len(train_loader)
    for i in train_loader:
        x = i[0].to(DEVICE)
        y = i[1].to(DEVICE)
        l = i[2]
        pred, hidden = model.forward(x)
        optimizer.zero_grad()
        loss = criterion(pred, y).to(DEVICE)
        train_loss.update(loss.item(),x.size(0))
        loss.backward()
        optimizer.step()
    return train_loss.avg
        
def validate(model, dev_loader, criterion, optimizer, epoch):
    b_size = DEV_BATCH_SIZE
    valid_loss = average_meter()
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        step = 0
        for i in dev_loader:
            x = i[0].to(DEVICE)
            y = i[1].to(DEVICE)
            l = i[2]
            pred, hidden = model.forward(x)
            loss = criterion(pred, y).to(DEVICE)
            valid_loss.update(loss.item(),x.size(0))
            pred = torch.max(pred, 1)[1]
            correct += (pred == y).float().sum()
            total += y.shape[0]
            step += 1
    return valid_loss.avg, correct/total

In [259]:
best_acc = 0
acc_model = None

for epoch in range(n_epochs):
    torch.cuda.empty_cache()
    train_loss = train(model, train_dataloader, criterion, optimizer, epoch)
    torch.cuda.empty_cache()
    scheduler.step()
    valid_loss, valid_acc = validate(model, dev_dataloader, criterion, optimizer, epoch)
    if valid_acc > best_acc:
        acc_model = deepcopy(model)
        best_acc = valid_acc
    print("{}%, Epoch {}, train loss {:.8f}, validate loss {:.8f}, acc {:.8f}".format(100*(epoch+1)/n_epochs,str(epoch).zfill(2),train_loss,valid_loss,valid_acc))

2.0%, Epoch 00, train loss 16.04284865, validate loss 24.87347794, acc 0.14800000
4.0%, Epoch 01, train loss 15.30215074, validate loss 9.10035419, acc 0.17000000
6.0%, Epoch 02, train loss 8.12217488, validate loss 6.56886101, acc 0.20000000
8.0%, Epoch 03, train loss 5.29472093, validate loss 4.67325211, acc 0.18600000
10.0%, Epoch 04, train loss 4.00887791, validate loss 3.56146979, acc 0.22400001
12.0%, Epoch 05, train loss 3.29977558, validate loss 3.17567205, acc 0.21800001
14.0%, Epoch 06, train loss 2.78575575, validate loss 2.64670539, acc 0.24400002
16.0%, Epoch 07, train loss 2.57165755, validate loss 2.47290254, acc 0.25400001
18.0%, Epoch 08, train loss 2.45154045, validate loss 2.29291654, acc 0.25800002
20.0%, Epoch 09, train loss 2.35066129, validate loss 2.43983483, acc 0.23000000
22.0%, Epoch 10, train loss 2.30753548, validate loss 2.18877149, acc 0.28400001
24.0%, Epoch 11, train loss 2.18666603, validate loss 2.22113991, acc 0.27400002
26.0%, Epoch 12, train loss 2

In [ ]:
class CNN_Text(nn.Module):
    
    def __init__(self):
        super(CNN_Text, self).__init__()
        filter_sizes = [2,3,4]
        num_filters = 100
        n_classes = 7
        self.convs1 = nn.ModuleList([nn.Conv2d(1, num_filters, (K, 201)) for K in filter_sizes])
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(len(filter_sizes)*num_filters, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 7)
        self.sigmoid = nn.Sigmoid()
        self.softmax = nn.Softmax()

    def forward(self, x):
        x = x.unsqueeze(1)  
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1] 
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  
        x = torch.cat(x, 1)
        x = self.dropout(x)  
        x = self.fc1(x)
        x = self.sigmoid(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

In [206]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
n_epochs = 50
model = CNN_Text().to(DEVICE)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

In [207]:
best_acc = 0
acc_model = None

for epoch in range(n_epochs):
    torch.cuda.empty_cache()
    train_loss = train(model, train_dataloader, criterion, optimizer, epoch)
    torch.cuda.empty_cache()
    scheduler.step()
    valid_loss, valid_acc = validate(model, dev_dataloader, criterion, optimizer, epoch)
    if valid_acc > best_acc:
        acc_model = deepcopy(model)
        best_acc = valid_acc
    print("{}%, Epoch {}, train loss {:.8f}, validate loss {:.8f}, acc {:.8f}".format(100*(epoch+1)/n_epochs,str(epoch).zfill(2),train_loss,valid_loss,valid_acc))

C:\APPs\Annaconda3\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


2.0%, Epoch 00, train loss 1.94474753, validate loss 1.93987871, acc 0.14400001
4.0%, Epoch 01, train loss 1.92040681, validate loss 1.88389413, acc 0.23800001
6.0%, Epoch 02, train loss 1.86053390, validate loss 1.83678313, acc 0.30400002
8.0%, Epoch 03, train loss 1.81620807, validate loss 1.80308235, acc 0.36800003
10.0%, Epoch 04, train loss 1.78283451, validate loss 1.77127900, acc 0.41200003
12.0%, Epoch 05, train loss 1.75160653, validate loss 1.75186491, acc 0.41000003
14.0%, Epoch 06, train loss 1.72311918, validate loss 1.73176570, acc 0.42600003
16.0%, Epoch 07, train loss 1.70071030, validate loss 1.71845526, acc 0.44200003
18.0%, Epoch 08, train loss 1.68143491, validate loss 1.70948460, acc 0.44600001
20.0%, Epoch 09, train loss 1.66665910, validate loss 1.70266666, acc 0.44600001
22.0%, Epoch 10, train loss 1.65328116, validate loss 1.69610053, acc 0.45600003
24.0%, Epoch 11, train loss 1.64310538, validate loss 1.69214867, acc 0.47200003
26.0%, Epoch 12, train loss 1.63